In [1]:
import os
import librosa
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.platform import gfile

tensorboard --logdir=/Users/erenmac/Desktop/ENGR497/Code/log



Variables for tuning:

batch size

iterations

learning rate

dropout

In [2]:
Path_Train ="/Users/erenmac/Desktop/ENGR497/Data/Speech_Data/speech_commands_v0.02/Train_Data/"
Path_Test = "/Users/erenmac/Desktop/ENGR497/Data/Speech_Data/speech_commands_v0.02/Test_Data/"
Path_Predict = "/Users/erenmac/Desktop/ENGR497/Data/Speech_Data/speech_commands_v0.01/1/"
Batch_Size = 35
Iterations =7500
Iterations_Test=200
Eval_Every = 5
Height = 20
Width = 440
Num_Labels=0
Learning_Rate = 1E-4
Logdir = "log/"
Label_To_Index_Map = {}

In [3]:
def init (path):
    labels = os.listdir(path)
    index = 0 
    for label in labels:
        if not label == ".DS_Store":
            Label_To_Index_Map[label] = index
            index +=1
    
    global Num_Labels
    Num_Labels = len(Label_To_Index_Map)

In [4]:
def one_hot_encoding(label):
    encoding = [0] * len(Label_To_Index_Map)
    encoding[Label_To_Index_Map[label]] = 1
    return encoding

In [5]:
def get_mfcc (wave_path, Pad_width = Width):
    wave, sr = librosa.load(wave_path,mono=True)
    mfccs = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=Height)
    mfccs = np.pad(mfccs,((0,0),(0, Pad_width - len(mfccs[0]))),mode = "constant")
    return mfccs

In [6]:
def get_batch(batch_size, path):
    x = []
    y = []
    random.seed(42)
    path = os.path.join(path,"*","*.wav")
    waves = gfile.Glob(path)
    while True:
        random.shuffle(waves)
        for wave_path in waves:
            _,label = os.path.split(os.path.dirname(wave_path))
            x.append(get_mfcc(wave_path))
            y.append(one_hot_encoding(label))
            if (len(x) == batch_size):
                yield x,y
                x=[]
                y=[]

In [7]:
def get_model(input,dropout):
    with tf.name_scope("Conv1"):
        input_4D = tf.reshape(input,[-1, Height,Width,1])
        w1 = tf.Variable(tf.truncated_normal([12,8,1,44],stddev = 0.01),name="W")
        b1 = tf.Variable(tf.zeros([44]),name = "B")
        conv1 = tf.nn.conv2d(input_4D, w1, strides=[1,1,1,1],padding = "SAME")
        act1 = tf.nn.relu(conv1 + b1)
        drop1 = tf.nn.dropout(act1,dropout)
        max_pooll = tf.nn.max_pool(drop1,ksize=[1,2,2,1],strides=[1,2,2,1],padding = "SAME")
        tf.summary.histogram("weights",w1)
        tf.summary.histogram("biases",b1)
        tf.summary.histogram("activations",act1)
        tf.summary.histogram("dropouts", drop1)
        
    with tf.name_scope("Conv2"):
        w2 = tf.Variable(tf.truncated_normal([6,4,44,44],stddev=0.01),name="W")
        b2 = tf.Variable(tf.zeros([44]),name ="B")
        conv2 = tf.nn.conv2d(max_pooll,w2,strides=[1,1,1,1],padding = "SAME")
        act2 = tf.nn.relu(conv2 + b2)
        drop2 = tf.nn.dropout(act2,dropout)
        max_pooll = tf.nn.max_pool(drop2,ksize=[1,2,2,1],strides=[1,2,2,1],padding = "SAME")
        tf.summary.histogram("weights",w2)
        tf.summary.histogram("biases",b2)
        tf.summary.histogram("activations",act2)
        tf.summary.histogram("dropouts", drop2)
        
    with tf.name_scope("Conv3"):
        w3 = tf.Variable(tf.truncated_normal([6,4,44,44],stddev= 0.01), name = "W")
        b3 = tf.Variable(tf.zeros([44]),name ="B")
        conv3 = tf.nn.conv2d(max_pooll,w3,strides=[1,1,1,1],padding = "SAME")
        act3 = tf.nn.relu(conv3 + b3)
        drop3 = tf.nn.dropout(act3,dropout)
        tf.summary.histogram("weights",w3)
        tf.summary.histogram("biases",b3)
        tf.summary.histogram("activations",act3)
        tf.summary.histogram("dropouts", drop3)
        
        conv_shape = drop3.get_shape()
        count = int(conv_shape[1] * conv_shape[2] * conv_shape[3])
        flat_output = tf.reshape(drop3, [-1,count])
    
    with tf.name_scope("FC"):
        w4= tf.Variable(tf.truncated_normal([count, Num_Labels], stddev = 0.01))
        b4= tf.Variable(tf.zeros([Num_Labels]))
        fc= tf.add(tf.matmul(flat_output,w4),b4)
        tf.summary.histogram("weights", w4)
        tf.summary.histogram("biases", b4)
        
    return fc

In [8]:
def main():
    print (Label_To_Index_Map)
    tf.reset_default_graph()
    sess = tf.Session()

    x = tf.placeholder(tf.float32, shape=[None, Height,Width], name = "input")
    y = tf.placeholder(tf.float32, shape=[None, Num_Labels], name="label")

    dropout = tf.placeholder(tf.float32, name="dropout")

    logits = get_model(x, dropout)

    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels = y), name = "loss")
        tf.summary.scalar("loss",loss)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(Learning_Rate).minimize(loss)

    with tf.name_scope("accuracy"):
        predicted = tf.argmax(logits,1)
        
        #Backpropagation  
        truth = tf.argmax(y,1)
        correct_prediction = tf.equal(predicted,truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        #Confusion matrix
        confusion_matrix = tf.confusion_matrix(truth, predicted, num_classes=Num_Labels)
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(Logdir)
    writer.add_graph(sess.graph)

    #Training Model
    print ("\nStarting training\n")
    batch = get_batch(Batch_Size,Path_Train)
    start_time = time.time()
    for i in range(1,Iterations+1):
        X, Y = next(batch)

        if i % Eval_Every == 0:
            [train_accuracy, train_loss, s] = sess.run([accuracy, loss, summ], feed_dict={x: X, y: Y, dropout: 0.5})
            acc_and_loss = [i, train_loss, train_accuracy *100]
            print ("Iteration # {}. Train Loss: {:.2f}. Train Acc: {:.0f}%".format(*acc_and_loss))
            writer.add_summary(s,i)

        if i% (Eval_Every * 20)== 0:
            train_confusion_matrix = sess.run([confusion_matrix], feed_dict = {x : X, y: Y, dropout:0.5})
            header = Label_To_Index_Map.keys()
            df=pd.DataFrame(np.reshape(train_confusion_matrix, (Num_Labels, Num_Labels)),columns = header, index = header)
            print ("\nConfusion Matrix:\n {}\n ".format(df))
            
        sess.run(train_step, feed_dict= {x: X, y: Y, dropout: 0.5})
        
    save_path = saver.save(sess, "/Users/erenmac/Desktop/ENGR497/Code/log/model.ckpt")
    print("Model saved in path: %s" % save_path)    
    print ("\nTotal training time: {:0f} seconds\n".format(time.time()- start_time))

    #Testing Model
    batch = get_batch (Batch_Size, Path_Test)
    total_accuracy = 0
    for i in range (Iterations_Test):
        X, Y = next (batch, Path_Test)

        #Testing
        test_accuracy, s = sess.run([accuracy, summ], feed_dict= {x : X, y: Y, dropout:1.0})
        print ("Iteration # {}. Test Accuracy: {:.0f}%".format (i+1,test_accuracy * 100))
        total_accuracy += (test_accuracy / Iterations_Test)

    print ("\nFinal Test Accuracy: {:.0f}%".format(total_accuracy * 100))
    print ("------------End Of Learning------------")

In [ ]:
if __name__ == "__main__":
    init(Path_Train)
    main()

{'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'backward': 4, 'learn': 5, 'bed': 6, 'happy': 7, 'go': 8, 'dog': 9, 'no': 10, 'wow': 11, 'follow': 12, 'nine': 13, 'left': 14, 'stop': 15, 'three': 16, 'sheila': 17, 'one': 18, 'bird': 19, 'zero': 20, 'seven': 21, 'up': 22, 'visual': 23, 'marvin': 24, 'two': 25, 'house': 26, 'down': 27, 'six': 28, 'yes': 29, 'on': 30, 'five': 31, 'forward': 32, 'off': 33, 'four': 34}

Starting training

Iteration # 5. Train Loss: 3.64. Train Acc: 3%
Iteration # 10. Train Loss: 3.51. Train Acc: 3%
Iteration # 15. Train Loss: 3.48. Train Acc: 3%
Iteration # 20. Train Loss: 3.52. Train Acc: 3%
Iteration # 25. Train Loss: 3.49. Train Acc: 0%
Iteration # 30. Train Loss: 3.50. Train Acc: 3%
Iteration # 35. Train Loss: 3.47. Train Acc: 0%
Iteration # 40. Train Loss: 3.44. Train Acc: 6%
Iteration # 45. Train Loss: 3.50. Train Acc: 6%
Iteration # 50. Train Loss: 3.47. Train Acc: 9%
Iteration # 55. Train Loss: 3.61. Train Acc: 3%
Iteration # 60. Train Loss: 3.58. Tra

Iteration # 205. Train Loss: 3.40. Train Acc: 3%
Iteration # 210. Train Loss: 3.42. Train Acc: 6%
Iteration # 215. Train Loss: 3.29. Train Acc: 6%
Iteration # 220. Train Loss: 3.38. Train Acc: 9%
Iteration # 225. Train Loss: 3.17. Train Acc: 9%
Iteration # 230. Train Loss: 3.41. Train Acc: 3%
Iteration # 235. Train Loss: 3.22. Train Acc: 14%
Iteration # 240. Train Loss: 3.17. Train Acc: 14%
Iteration # 245. Train Loss: 3.19. Train Acc: 6%
Iteration # 250. Train Loss: 3.00. Train Acc: 9%
Iteration # 255. Train Loss: 3.17. Train Acc: 17%
Iteration # 260. Train Loss: 3.00. Train Acc: 23%
Iteration # 265. Train Loss: 3.18. Train Acc: 9%
Iteration # 270. Train Loss: 3.06. Train Acc: 14%
Iteration # 275. Train Loss: 3.13. Train Acc: 11%
Iteration # 280. Train Loss: 3.35. Train Acc: 9%
Iteration # 285. Train Loss: 2.66. Train Acc: 31%
Iteration # 290. Train Loss: 3.10. Train Acc: 11%
Iteration # 295. Train Loss: 2.76. Train Acc: 26%
Iteration # 300. Train Loss: 2.98. Train Acc: 14%

Confusion

Iteration # 405. Train Loss: 2.76. Train Acc: 29%
Iteration # 410. Train Loss: 3.00. Train Acc: 20%
Iteration # 415. Train Loss: 2.60. Train Acc: 17%
Iteration # 420. Train Loss: 2.30. Train Acc: 26%
Iteration # 425. Train Loss: 2.53. Train Acc: 34%
Iteration # 430. Train Loss: 2.52. Train Acc: 29%
Iteration # 435. Train Loss: 2.29. Train Acc: 34%
Iteration # 440. Train Loss: 3.02. Train Acc: 14%
Iteration # 445. Train Loss: 2.50. Train Acc: 29%
Iteration # 450. Train Loss: 2.53. Train Acc: 20%
Iteration # 455. Train Loss: 3.06. Train Acc: 9%
Iteration # 460. Train Loss: 2.65. Train Acc: 23%
Iteration # 465. Train Loss: 2.48. Train Acc: 31%
Iteration # 470. Train Loss: 2.57. Train Acc: 14%
Iteration # 475. Train Loss: 2.68. Train Acc: 31%
Iteration # 480. Train Loss: 2.57. Train Acc: 23%
Iteration # 485. Train Loss: 2.50. Train Acc: 29%
Iteration # 490. Train Loss: 2.58. Train Acc: 26%
Iteration # 495. Train Loss: 2.30. Train Acc: 31%
Iteration # 500. Train Loss: 2.47. Train Acc: 31%



Iteration # 605. Train Loss: 2.34. Train Acc: 37%
Iteration # 610. Train Loss: 2.17. Train Acc: 31%
Iteration # 615. Train Loss: 2.61. Train Acc: 34%
Iteration # 620. Train Loss: 1.93. Train Acc: 49%
Iteration # 625. Train Loss: 2.45. Train Acc: 37%
Iteration # 630. Train Loss: 2.29. Train Acc: 37%
Iteration # 635. Train Loss: 2.15. Train Acc: 31%
Iteration # 640. Train Loss: 1.90. Train Acc: 43%
Iteration # 645. Train Loss: 2.22. Train Acc: 49%
Iteration # 650. Train Loss: 1.97. Train Acc: 49%
Iteration # 655. Train Loss: 2.12. Train Acc: 40%
Iteration # 660. Train Loss: 2.47. Train Acc: 31%
Iteration # 665. Train Loss: 2.03. Train Acc: 37%
Iteration # 670. Train Loss: 2.44. Train Acc: 17%
Iteration # 675. Train Loss: 2.16. Train Acc: 43%
Iteration # 680. Train Loss: 2.13. Train Acc: 29%
Iteration # 685. Train Loss: 2.01. Train Acc: 43%
Iteration # 690. Train Loss: 2.22. Train Acc: 31%
Iteration # 695. Train Loss: 2.81. Train Acc: 23%
Iteration # 700. Train Loss: 2.24. Train Acc: 34%


Iteration # 805. Train Loss: 2.05. Train Acc: 40%
Iteration # 810. Train Loss: 2.64. Train Acc: 29%
Iteration # 815. Train Loss: 2.34. Train Acc: 29%
Iteration # 820. Train Loss: 1.98. Train Acc: 43%
Iteration # 825. Train Loss: 2.43. Train Acc: 31%
Iteration # 830. Train Loss: 2.33. Train Acc: 34%
Iteration # 835. Train Loss: 2.07. Train Acc: 46%
Iteration # 840. Train Loss: 1.96. Train Acc: 49%
Iteration # 845. Train Loss: 2.31. Train Acc: 26%
Iteration # 850. Train Loss: 1.93. Train Acc: 57%
Iteration # 855. Train Loss: 1.76. Train Acc: 46%
Iteration # 860. Train Loss: 2.22. Train Acc: 40%
Iteration # 865. Train Loss: 2.25. Train Acc: 34%
Iteration # 870. Train Loss: 1.72. Train Acc: 49%
Iteration # 875. Train Loss: 1.93. Train Acc: 43%
Iteration # 880. Train Loss: 2.16. Train Acc: 31%
Iteration # 885. Train Loss: 2.11. Train Acc: 37%
Iteration # 890. Train Loss: 2.11. Train Acc: 37%
Iteration # 895. Train Loss: 2.40. Train Acc: 31%
Iteration # 900. Train Loss: 2.17. Train Acc: 34%


...

Iteration # 7405. Train Loss: 0.31. Train Acc: 91%
Iteration # 7410. Train Loss: 0.54. Train Acc: 83%
Iteration # 7415. Train Loss: 0.64. Train Acc: 83%
Iteration # 7420. Train Loss: 0.69. Train Acc: 77%
Iteration # 7425. Train Loss: 0.53. Train Acc: 86%
Iteration # 7430. Train Loss: 0.52. Train Acc: 89%
Iteration # 7435. Train Loss: 0.69. Train Acc: 80%
Iteration # 7440. Train Loss: 0.74. Train Acc: 80%
Iteration # 7445. Train Loss: 0.58. Train Acc: 83%
Iteration # 7450. Train Loss: 0.41. Train Acc: 89%
Iteration # 7455. Train Loss: 0.62. Train Acc: 74%
Iteration # 7460. Train Loss: 0.48. Train Acc: 86%
Iteration # 7465. Train Loss: 0.47. Train Acc: 86%
Iteration # 7470. Train Loss: 0.64. Train Acc: 83%
Iteration # 7475. Train Loss: 0.75. Train Acc: 71%
Iteration # 7480. Train Loss: 0.56. Train Acc: 83%
Iteration # 7485. Train Loss: 0.98. Train Acc: 74%
Iteration # 7490. Train Loss: 0.82. Train Acc: 69%
Iteration # 7495. Train Loss: 0.58. Train Acc: 83%
Iteration # 7500. Train Loss: 0

Iteration # 48. Test Accuracy: 94%
Iteration # 49. Test Accuracy: 89%
Iteration # 50. Test Accuracy: 91%
Iteration # 51. Test Accuracy: 89%
Iteration # 52. Test Accuracy: 83%
Iteration # 53. Test Accuracy: 89%
Iteration # 54. Test Accuracy: 89%
Iteration # 55. Test Accuracy: 94%
Iteration # 56. Test Accuracy: 86%
Iteration # 57. Test Accuracy: 83%
Iteration # 58. Test Accuracy: 86%
Iteration # 59. Test Accuracy: 91%
Iteration # 60. Test Accuracy: 91%
Iteration # 61. Test Accuracy: 94%
Iteration # 62. Test Accuracy: 94%
Iteration # 63. Test Accuracy: 89%
Iteration # 64. Test Accuracy: 83%
Iteration # 65. Test Accuracy: 80%
Iteration # 66. Test Accuracy: 71%
Iteration # 67. Test Accuracy: 77%
Iteration # 68. Test Accuracy: 77%
Iteration # 69. Test Accuracy: 94%
Iteration # 70. Test Accuracy: 80%
Iteration # 71. Test Accuracy: 89%
Iteration # 72. Test Accuracy: 86%
Iteration # 73. Test Accuracy: 89%
Iteration # 74. Test Accuracy: 86%
Iteration # 75. Test Accuracy: 89%
Iteration # 76. Test

Iteration # 277. Test Accuracy: 91%
Iteration # 278. Test Accuracy: 86%
Iteration # 279. Test Accuracy: 89%
Iteration # 280. Test Accuracy: 80%
Iteration # 281. Test Accuracy: 91%
Iteration # 282. Test Accuracy: 80%
Iteration # 283. Test Accuracy: 94%
Iteration # 284. Test Accuracy: 74%
Iteration # 285. Test Accuracy: 91%
Iteration # 286. Test Accuracy: 86%
Iteration # 287. Test Accuracy: 86%
Iteration # 288. Test Accuracy: 71%
Iteration # 289. Test Accuracy: 86%
Iteration # 290. Test Accuracy: 77%
Iteration # 291. Test Accuracy: 89%
Iteration # 292. Test Accuracy: 86%
Iteration # 293. Test Accuracy: 74%
Iteration # 294. Test Accuracy: 69%
Iteration # 295. Test Accuracy: 80%
Iteration # 296. Test Accuracy: 80%
Iteration # 297. Test Accuracy: 77%
Iteration # 298. Test Accuracy: 77%
Iteration # 299. Test Accuracy: 97%
Iteration # 300. Test Accuracy: 91%
Iteration # 301. Test Accuracy: 86%
Iteration # 302. Test Accuracy: 89%
Iteration # 303. Test Accuracy: 86%
Iteration # 304. Test Accura

Iteration # 505. Test Accuracy: 94%
Iteration # 506. Test Accuracy: 86%
Iteration # 507. Test Accuracy: 89%
Iteration # 508. Test Accuracy: 86%
Iteration # 509. Test Accuracy: 86%
Iteration # 510. Test Accuracy: 89%
Iteration # 511. Test Accuracy: 89%
Iteration # 512. Test Accuracy: 89%
Iteration # 513. Test Accuracy: 89%
Iteration # 514. Test Accuracy: 89%
Iteration # 515. Test Accuracy: 89%
Iteration # 516. Test Accuracy: 94%
Iteration # 517. Test Accuracy: 86%
Iteration # 518. Test Accuracy: 80%
Iteration # 519. Test Accuracy: 80%
Iteration # 520. Test Accuracy: 91%
Iteration # 521. Test Accuracy: 89%
Iteration # 522. Test Accuracy: 89%
Iteration # 523. Test Accuracy: 89%
Iteration # 524. Test Accuracy: 89%
Iteration # 525. Test Accuracy: 94%
Iteration # 526. Test Accuracy: 86%
Iteration # 527. Test Accuracy: 80%
Iteration # 528. Test Accuracy: 83%
Iteration # 529. Test Accuracy: 80%
Iteration # 530. Test Accuracy: 89%
Iteration # 531. Test Accuracy: 77%
Iteration # 532. Test Accura

Iteration # 733. Test Accuracy: 86%
Iteration # 734. Test Accuracy: 80%
Iteration # 735. Test Accuracy: 89%
Iteration # 736. Test Accuracy: 71%
Iteration # 737. Test Accuracy: 91%
Iteration # 738. Test Accuracy: 86%
Iteration # 739. Test Accuracy: 86%
Iteration # 740. Test Accuracy: 83%
Iteration # 741. Test Accuracy: 83%
Iteration # 742. Test Accuracy: 91%
Iteration # 743. Test Accuracy: 94%
Iteration # 744. Test Accuracy: 89%
Iteration # 745. Test Accuracy: 86%
Iteration # 746. Test Accuracy: 86%
Iteration # 747. Test Accuracy: 77%


In [9]:
def loadmodel ():
    logadress = "/Users/erenmac/Desktop/ENGR497/Code/log/1"
    init(Path_Train)
    keylist = []
    for key in Label_To_Index_Map:
        keylist.append(key)
    print (keylist)
    tf.reset_default_graph()
    sess = tf.Session()

    x = tf.placeholder(tf.float32, shape=[None, Height,Width], name = "input")
    dropout = tf.placeholder(tf.float32, name="dropout")
    logits = get_model(x, dropout)

    with tf.name_scope("predict"):
        predict_data = tf.argmax(logits,1)


    saver = tf.train.Saver()    
    # Restore variables from disk.
    saver.restore(sess, logadress + "/model.ckpt")
    print("Model restored.")
        
        
    stopcond = True
    while stopcond == True:
        filename_input = input("Predict File Path Name:")
        if filename_input == "False":
            stopcond = False
        else:
            mfcc_pred = get_mfcc(Path_Predict + filename_input + ".wav")
            mfcc_pred_list = []
            mfcc_pred_list.append(mfcc_pred)
            Prediction = sess.run([predict_data], feed_dict= {x : mfcc_pred_list, dropout: 1.0})
            for word, label in Label_To_Index_Map.items():
                if label == Prediction[0]:
                    print("You said:")
                    print(word)

In [ ]:
loadmodel()

['right', 'eight', 'cat', 'tree', 'backward', 'learn', 'bed', 'happy', 'go', 'dog', 'no', 'wow', 'follow', 'nine', 'left', 'stop', 'three', 'sheila', 'one', 'bird', 'zero', 'seven', 'up', 'visual', 'marvin', 'two', 'house', 'down', 'six', 'yes', 'on', 'five', 'forward', 'off', 'four']
INFO:tensorflow:Restoring parameters from /Users/erenmac/Desktop/ENGR497/Code/log/model.ckpt
Model restored.
